In [ ]:
import pandas as pd

In [14]:
ahis = pd.read_csv('63_ahis.csv', index_col=0)
ahis.set_index('mesa_id', inplace=True)
subject_infos = pd.read_csv('/vol/sleepstudy/datasets/mesa/datasets/mesa-sleep-harmonized-dataset-0.7.0.csv')
subject_infos.set_index('mesaid', inplace=True)

In [16]:
age_list = []
bmi_list = []
sex_list = []
cur_smoker_list = []
ever_smoker_list = []
pred_ahi_list = []
true_ahi_list = []

for mesa_id in ahis.index.values:
    subject_info = subject_infos.loc[mesa_id]
    age_list.append(subject_info['nsrr_age'])
    bmi_list.append(subject_info['nsrr_bmi'])
    sex_list.append(0 if subject_info['nsrr_sex'] == 'female' else 1)
    cur_smoker_list.append(-1 if subject_info['nsrr_current_smoker'] == 'no' else (1 if subject_info['nsrr_ever_smoker'] == 'yes' else 0))
    ever_smoker_list.append(-1 if subject_info['nsrr_ever_smoker'] == 'no' else (1 if subject_info['nsrr_ever_smoker'] == 'yes' else 0))
    pred_ahi_list.append(ahis['ahi_pred'][mesa_id])
    true_ahi_list.append(ahis['ahi_true'][mesa_id])

In [22]:
pd.DataFrame({
    'age': age_list,
    'bmi': bmi_list,
    'sex': sex_list,
    'cur_smoker': cur_smoker_list,
    'ever_smoker': ever_smoker_list,
    'pred_ahi': pred_ahi_list,
    'true_ahi': true_ahi_list
}).to_csv('63_ahi_dataset.csv', index=False)

In [25]:
full_dataset = pd.read_csv('63_ahi_dataset.csv', index_col=False)

In [26]:
full_dataset

,age,bmi,sex,cur_smoker,ever_smoker,pred_ahi,true_ahi
0,85.0,20.00,0,0,-1,11.860465,12.589286
1,84.0,24.55,1,0,-1,39.553903,43.155452
2,67.0,20.36,0,0,-1,16.349614,20.818966
3,63.0,27.32,1,-1,1,11.489362,10.757342
4,63.0,47.72,0,1,1,45.014138,96.354680
...,...,...,...,...,...,...,...
1875,65.0,25.75,1,0,-1,59.610895,51.687243
1876,82.0,24.40,1,-1,0,7.888112,22.500000
1877,79.0,27.89,1,-1,1,2.589928,4.541485
1878,58.0,33.86,1,0,-1,38.771060,47.155963


In [27]:
def get_fold(full_dataset, fold_nr):
    ds_len = len(full_dataset)
    fold_len = ds_len // 4
    folds = [
        full_dataset[:fold_len],
        full_dataset[fold_len:fold_len*2],
        full_dataset[fold_len*2:fold_len*3],
        full_dataset[fold_len*3:]
    ]
    train_dataset = pd.concat(folds[:fold_nr] + folds[fold_nr+1:])
    test_dataset = folds[fold_nr]
    return train_dataset, test_dataset

In [33]:
get_fold(full_dataset, 3)[1].values

array([[63.        , 29.07      ,  0.        , ..., -1.        ,
        14.21538462, 10.176     ],
       [79.        , 28.23      ,  1.        , ...,  1.        ,
        39.81755986, 44.02649007],
       [64.        , 32.36      ,  1.        , ..., -1.        ,
        22.63027295, 22.14716526],
       ...,
       [79.        , 27.89      ,  1.        , ...,  1.        ,
         2.58992806,  4.54148472],
       [58.        , 33.86      ,  1.        , ..., -1.        ,
        38.77106046, 47.1559633 ],
       [64.        , 22.51      ,  0.        , ...,  1.        ,
        11.3814433 , 19.31034483]])

In [36]:
get_fold(full_dataset, 3)[1].values[-1]

array([64.        , 22.51      ,  0.        , -1.        ,  1.        ,
       11.3814433 , 19.31034483])

In [39]:
get_fold(full_dataset, 3)[1][['age', 'bmi', 'sex', 'cur_smoker', 'ever_smoker', 'pred_ahi']].values

array([[63.        , 29.07      ,  0.        ,  0.        , -1.        ,
        14.21538462],
       [79.        , 28.23      ,  1.        , -1.        ,  1.        ,
        39.81755986],
       [64.        , 32.36      ,  1.        ,  0.        , -1.        ,
        22.63027295],
       ...,
       [79.        , 27.89      ,  1.        , -1.        ,  1.        ,
         2.58992806],
       [58.        , 33.86      ,  1.        ,  0.        , -1.        ,
        38.77106046],
       [64.        , 22.51      ,  0.        , -1.        ,  1.        ,
        11.3814433 ]])

In [41]:
len(get_fold(full_dataset, 3)[1]['true_ahi'].values)

470

In [46]:
len(np.concat([get_fold(full_dataset, 3)[1]['true_ahi'].values, get_fold(full_dataset, 3)[1]['true_ahi'].values], axis=1))

AxisError: axis 1 is out of bounds for array of dimension 1